In [ ]:
import pathlib
import sys

In [ ]:
import pandas as pd

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
raw_dir_path = data_dir_path / "raw"
sample_submission_path = raw_dir_path / "sample_submission.csv"
models_dir_path = root_dir_path / "models"
prediction_path = models_dir_path / "prediction.parquet"
submission_path = models_dir_path / "submission.csv.gz"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *

In [ ]:
sample_submission = pd.read_csv(sample_submission_path)

In [ ]:
prediction = pd.read_parquet(prediction_path)

In [ ]:
train_days = 1913
test_days = 28

In [ ]:
d = prediction["d"].str[2:]
d = d.astype("int32")
d -= train_days + 1

is_evaluation = d >= test_days
d %= test_days

prediction["d"] = d
prediction.loc[is_evaluation, "id"] = prediction.loc[is_evaluation, "id"].str.replace(
    "_validation", "_evaluation"
)

In [ ]:
submission = pd.pivot(prediction, index="id", columns="d", values=target)

In [ ]:
submission.reset_index(inplace=True)

In [ ]:
submission.columns = ["id"] + [f"F{i + 1}" for i in range(test_days)]

In [ ]:
submission = pd.merge(sample_submission["id"], submission, how="left")

In [ ]:
submission.to_csv(submission_path, index=False)